In [1]:
import monai
import os
from glob import glob
import torch 
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    ToTensord,
    Spacingd,
    RandAffined,
    RandRotated,
    Orientationd,
    RandGaussianNoised,
     CropForeground,
    ScaleIntensityRanged,
    EnsureChannelFirst,
    MapTransform,
   )
from monai.data import DataLoader, Dataset
from monai.utils import first
import matplotlib.pyplot as plt

In [2]:
data_dir=r'C:\Users\Abhay Tiwari\DATASET\Data_Train_Test/'

In [3]:
train_images = sorted(glob(os.path.join(data_dir, 'TrainVolumes', '*.nii.gz'))) 
train_labels = sorted(glob(os.path.join(data_dir, 'TrainSegmentation', '*.nii.gz'))) 

val_images = sorted(glob(os.path.join(data_dir, 'TestVolumes', '*.nii.gz'))) 
val_labels = sorted(glob(os.path.join(data_dir, 'TestSegmentation', '*.nii.gz'))) 
#print(train_labels)
#print(val_labels)

train_files = [{"image":image_name, 'label':label_name} for image_name, label_name in zip(train_images, train_labels)] 
val_files = [{"image":image_name, 'label':label_name} for image_name, label_name in zip(val_images, val_labels)] 


In [5]:
original_transforms = Compose(

  [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        ToTensord(keys=["image", "label"]),
  ]
)


train_transforms = Compose(

  [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        Spacingd(keys=["image"], pixdim=(1.5, 1.5, 2.0)),
        ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True,), 
        CropForeground(keys=["image", "label"],source_key="image"),
        ToTensord(keys=["image", "label"]),
  ]
)

val_transforms = Compose(

 [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True,),
        ToTensord(keys=["image", "label"]),
  ]
)

C:\Users\Abhay Tiwari\AppData\Roaming\Python\Python39\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
C:\Users\Abhay Tiwari\AppData\Roaming\Python\Python39\site-packages\monai\utils\deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
C:\Users\Abhay Tiwari\AppData\Roaming\Python\Python39\site-packages\monai\utils\deprecate_utils.py:221: FutureWarning: monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta

In [6]:
original_ds = Dataset(data=train_files, transform=original_transforms)
original_loader = DataLoader(original_ds, batch_size=1)

train_ds = Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1)

val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1)  

In [ ]:
test_patient=first(train_loader)
original_patient=first(original_loader)
#print(test_patient)

In [ ]:
plt.figure('test', (12, 6)) 

plt.subplot(1, 3, 1) 
plt.title('slice of a patient') 
plt.imshow(original_patient['image'][0,0,:,:,59], cmap= "gray") 

plt.subplot(1, 3, 2) 
plt.title('slice of a patient') 
plt.imshow(test_patient['image'][0,0,:,:,59], cmap= "gray") 

plt.subplot(1,3,3) 
plt.title('label of a patient')
plt.imshow(test_patient['label'][0,0,:,:,59])
plt.show